In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
import threading
import os

class ReportApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Преобразование данных")
        self.root.geometry("800x600")
        
        self.main_file = tk.StringVar()
        self.sku_file = tk.StringVar()
        self.tt_file = tk.StringVar()
        
        self.create_widgets()
        
    def create_widgets(self):
        file_frame = ttk.LabelFrame(self.root, text="Выбор файлов")
        file_frame.pack(padx=10, pady=10, fill="x")
        
        ttk.Label(file_frame, text="Исходная таблица:").grid(row=0, column=0, sticky="w")
        ttk.Entry(file_frame, textvariable=self.main_file, width=50).grid(row=0, column=1)
        ttk.Button(file_frame, text="Обзор", command=lambda: self.select_file(self.main_file)).grid(row=0, column=2)
        
        ttk.Label(file_frame, text="Справочник SKU:").grid(row=1, column=0, sticky="w")
        ttk.Entry(file_frame, textvariable=self.sku_file, width=50).grid(row=1, column=1)
        ttk.Button(file_frame, text="Обзор", command=lambda: self.select_file(self.sku_file)).grid(row=1, column=2)
        
        ttk.Label(file_frame, text="Справочник ТТ:").grid(row=2, column=0, sticky="w")
        ttk.Entry(file_frame, textvariable=self.tt_file, width=50).grid(row=2, column=1)
        ttk.Button(file_frame, text="Обзор", command=lambda: self.select_file(self.tt_file)).grid(row=2, column=2)
        
        self.run_btn = ttk.Button(self.root, text="Создать отчет", command=self.start_processing)
        self.run_btn.pack(pady=10)
        
        self.progress = ttk.Progressbar(self.root, orient="horizontal", length=300, mode="determinate")
        self.progress.pack(pady=5)
        
        self.log_text = tk.Text(self.root, height=15, state="disabled")
        self.log_text.pack(padx=10, pady=10, fill="both", expand=True)
        
    def select_file(self, target_var):
        file_path = filedialog.askopenfilename(
            filetypes=[
                ("Excel files", "*.xlsx *.xlsb"), 
                ("XLSB files", "*.xlsb"), 
                ("XLSX files", "*.xlsx"),
                ("All files", "*.*")
            ]
        )
        if file_path:
            target_var.set(file_path)
            
    def log_message(self, message):
        self.log_text.config(state="normal")
        self.log_text.insert("end", message + "\n")
        self.log_text.see("end")
        self.log_text.config(state="disabled")
    
    def update_progress(self, value):
        self.progress["value"] = value
        self.root.update_idletasks()
        
    def start_processing(self):
        if not all([self.main_file.get(), self.sku_file.get(), self.tt_file.get()]):
            messagebox.showerror("Ошибка", "Необходимо выбрать все файлы!")
            return
            
        thread = threading.Thread(target=self.create_report)
        thread.start()
        
    def read_excel_file(self, file_path):
        try:
            _, ext = os.path.splitext(file_path)
            
            if ext.lower() == '.xlsb':
                return pd.read_excel(file_path, engine='pyxlsb')
            else:
                return pd.read_excel(file_path)
            
        except Exception as e:
            self.log_message(f"Ошибка чтения файла {file_path}: {str(e)}")
            raise
        
    def create_report(self):
        try:
            self.run_btn.config(state="disabled")
            self.log_message("Начало обработки данных...")
            
            # Загрузка данных
            self.log_message("Загрузка основной таблицы...")
            df_main = self.read_excel_file(self.main_file.get())
            self.update_progress(20)
            
            self.log_message("Загрузка справочника SKU...")
            df_sku = self.read_excel_file(self.sku_file.get())
            self.update_progress(40)
            
            self.log_message("Загрузка справочника ТТ...")
            df_tt = self.read_excel_file(self.tt_file.get())
            self.update_progress(60)
            
            # Обработка данных
            self.log_message("Обработка данных...")
            df_main['Себестоимость(руб.)'] = df_main['Цена вх.'] * df_main['Реализация шт.(кг)']
            df_main['Месяц'] = 1
            df_main['Год'] = 2025
            
            numeric_cols = ['Реализация шт.(кг)', 'Реализация Сумма', 'Себестоимость(руб.)']
            group_cols = ['Месяц', 'Год', '№ магазина', 'Код товара', 'Наименование товара']
            df_month = df_main.groupby(group_cols)[numeric_cols].sum().reset_index()
            
            df_month['Сеть'] = 'VERNYY'
            df_month['Глобальный код ТТ'] = 'VERNYY' + df_month['№ магазина'].astype(str)
            df_month['Глобальный Код SKU'] = 'VERNYY' + df_month['Код товара'].astype(str)
            
            sku_mapping = df_sku.set_index('Глобальный Код SKU')['Наименование SKU'].to_dict()
            df_month['Наименование SKU'] = df_month['Глобальный Код SKU'].map(sku_mapping)
            
            result_cols = [
                'Месяц', 'Год', 'Сеть', 'Глобальный код ТТ', 
                '№ магазина', 'Глобальный Код SKU', 'Наименование SKU',
                'Реализация шт.(кг)', 'Реализация Сумма', 'Себестоимость(руб.)'
            ]

            final_df = df_month[result_cols].rename(columns={
                '№ магазина': 'Код ТТ',
                'Реализация шт.(кг)': 'Продажа(шт)',
                'Реализация Сумма': 'Оборот(руб.)'
            })
            
            # Сохранение с форматированием
            self.log_message("Создание Excel с форматированием...")
            save_path = filedialog.asksaveasfilename(
                defaultextension=".xlsx",
                filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")]
            )
            
            if save_path:
                wb = Workbook()
                ws = wb.active
                ws.title = "Отчет VERNYY"
                
                # Запись данных
                for r in dataframe_to_rows(final_df, index=False, header=True):
                    ws.append(r)
                
                # Стили оформления
                header_font = Font(bold=True, color="FFFFFF")
                header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
                thin_border = Border(
                    left=Side(style='thin'), 
                    right=Side(style='thin'), 
                    top=Side(style='thin'), 
                    bottom=Side(style='thin')
                )
                
                # Форматирование заголовков
                for cell in ws[1]:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.border = thin_border
                    cell.alignment = Alignment(horizontal="center", vertical="center")
                
                # Форматирование данных
                for row in ws.iter_rows(min_row=2):
                    for cell in row:
                        cell.border = thin_border
                        cell.alignment = Alignment(horizontal="center", vertical="center")
                        
                        # Форматы чисел
                        if cell.column_letter in ['H', 'I', 'J']:
                            cell.number_format = '#,##0.00'
                        elif cell.column_letter in ['A', 'B']:
                            cell.number_format = '0'
                
                # Автоподбор ширины столбцов
                for col in ws.columns:
                    max_length = 0
                    column = col[0].column_letter
                    for cell in col:
                        try:
                            value_len = len(str(cell.value))
                            if value_len > max_length:
                                max_length = value_len
                        except:
                            pass
                    adjusted_width = (max_length + 2) * 1.2
                    ws.column_dimensions[column].width = adjusted_width
                
                # Фиксация заголовков
                ws.freeze_panes = 'A2'
                
                wb.save(save_path)
                self.log_message(f"Файл успешно сохранен: {save_path}")
                messagebox.showinfo("Готово", "Отчет сформирован с форматированием!")
            
            self.update_progress(100)
            
        except Exception as e:
            self.log_message(f"Ошибка: {str(e)}")
            messagebox.showerror("Ошибка", str(e))
        finally:
            self.run_btn.config(state="normal")
            self.update_progress(0)

if __name__ == "__main__":
    root = tk.Tk()
    app = ReportApp(root)
    root.mainloop()

2025-04-01 15:01:31.979 python[95117:7716146] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-01 15:01:31.979 python[95117:7716146] +[IMKInputSession subclass]: chose IMKInputSession_Modern
